# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [1]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [3]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9444444444444444

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [26]:
## Prueba 1 creada  por mi

import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import Bosque

def f1_objective(parametros):
    n_estimators, max_depth = parametros
    modelo, f1 = Bosque.RegresionBosque(X, y, int(n_estimators), int(max_depth))
    return f1

n_estimators_values = np.linspace(5, 50, 10)
max_depth_values = np.linspace(2, 10, 9)

X_train = np.random.uniform([5, 2], [50, 10], size=(5, 2))
y_train = np.array([f1_objective(x) for x in X_train])

kernel = 1.0 * RBF(length_scale=1)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gp.fit(X_train, y_train)

n = 30
f1_scores = []

for i in range(n):
    X= np.array(np.meshgrid(n_estimators_values, max_depth_values)).T.reshape(-1, 2)
    y_pred, y_std = gp.predict(X, return_std=True)
    next_point = X[np.argmax(y_std)]
    next_value = f1_objective(next_point)
    X_train = np.vstack([X_train, next_point])
    y_train = np.append(y_train, next_value)
    gp.fit(X_train, y_train)
    f1_scores.append(-y_train.min())

plt.plot(f1_scores)
plt.show()



ValueError: Found input variables with inconsistent numbers of samples: [90, 150]

In [12]:
#Prueba 2 hecha con IA

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from scipy.optimize import minimize

def f1_objective(parametros):
    n_estimators, max_depth = parametros
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60, random_state=10)
    mRF = RandomForestClassifier(n_estimators=int(n_estimators), max_depth=int(max_depth), random_state=10)
    mRF.fit(X_train, y_train)

    y_pred = mRF.predict(X_test)

    f1 = metrics.f1_score(y_test, y_pred, average="micro")

    return -f1

bounds = np.array([[5, 50],
                   [2, 10]])

X_train = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(5, 2))
y_train = np.array([f1_objective(x) for x in X_train])
kernel = 1.0 * RBF(length_scale=1)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gp.fit(X_train, y_train)

n = 30

for i in range(n):
    X_grid = np.linspace(bounds[0, 0], bounds[0, 1], 50).reshape(-1, 1)
    X_grid = np.hstack([X_grid, np.linspace(bounds[1, 0], bounds[1, 1], 50).reshape(-1, 1)])

    y_pred, y_std = gp.predict(X_grid, return_std=True)

    next_point = X_grid[np.argmax(y_std)]

    next_value = f1_objective(next_point)

    X_train = np.vstack([X_train, next_point])
    y_train = np.append(y_train, next_value)

    gp.fit(X_train, y_train)
    if (i + 1) % 5 == 0:
        print(f"Iteración {i+1}/{n}: Mejor F1 score = {-y_train.min():.4f}")

best_params_idx = np.argmin(y_train)
best_params = X_train[best_params_idx]
print(f"\nMejores parámetros encontrados: \nNúmero de árboles: {int(best_params[0])}, Profundidad: {int(best_params[1])}")
print(f"Mejor F1 score: {-y_train.min():.4f}")


Iteración 5/30: Mejor F1 score = 0.9556
Iteración 10/30: Mejor F1 score = 0.9556
Iteración 15/30: Mejor F1 score = 0.9667
Iteración 20/30: Mejor F1 score = 0.9667


/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Iteración 25/30: Mejor F1 score = 0.9667
Iteración 30/30: Mejor F1 score = 0.9667

Mejores parámetros encontrados: 
Número de árboles: 16, Profundidad: 3
Mejor F1 score: 0.9667


### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.